In [1]:
# Import the dependecies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Import the requests library.
import requests

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)
# Import the API key.
from config import weather_api_key

In [3]:
# Create a set of random latitudes and longitudes combinations
lats = np.random.uniform(low=-90.000,high=90.000,size=2000)
lngs = np.random.uniform(low=-180.000,high=180.000,size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

729

In [7]:
# Import the date time module from the datetime library
from datetime import datetime

In [8]:
url = "http://api.openweathermap.org/data/2.5/weather?units=imperial&APPID=" + weather_api_key

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
       # city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description":city_weather_description})

# If an error is experienced, skip the city.
    except:
        #print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")

Beginning Data Retrieval     
-----------------------------
-----------------------------
Data Retrieval Complete      


In [11]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Camana,-16.62,-72.71,59.86,82,88,8.66,PE,overcast clouds
1,Atuona,-9.80,-139.03,78.37,77,16,14.97,PF,few clouds
2,Bose,45.46,8.01,66.99,78,95,5.55,IT,overcast clouds
3,Hofn,64.25,-15.21,44.60,93,90,3.36,IS,overcast clouds
4,Houma,29.60,-90.72,80.01,83,1,3.36,US,clear sky
5,Saldanha,-33.01,17.94,48.20,87,5,1.12,ZA,clear sky
6,Huarmey,-10.07,-78.15,60.39,85,13,8.75,PE,few clouds
7,Padang,-0.95,100.35,83.73,64,59,2.42,ID,broken clouds
8,Hilo,19.73,-155.09,84.20,66,90,6.93,US,overcast clouds
9,Babakan,-7.28,107.22,80.89,59,2,2.93,ID,clear sky


In [12]:
new_column_order = ["City","Country","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Current Description"]
city_data_df = city_data_df[new_column_order]

In [13]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
#city_data_df.to_csv(output_data_file, index_label="City_ID")
city_data_df.to_csv(output_data_file)